In [1]:
! pip install transformers datasets peft accelerate bitsandbytes
! pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.0/12.0 MB 89.6 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 566.1/566.1 kB 67.8 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 198.9 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 234.3 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 556.4/556.4 kB 67.9 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.1/59.1 MB 243.0 MB/s  0:00:00m0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 276.4 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.5/803.5 kB 107.9 MB/s  0:00:00
  Attempting uninstall: fsspec━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━  3/15 [pyarrow]
    Found existing installation: fsspec 2025.12.0━━━━━━━━━━━━━  3/15 [pyarrow]
    Uninstalling fsspec-2025.12.0:━━━━━━━━━━━━━━━━━━━━━━━━━━━━  3/15 [pyarrow]
      Successfully uninstalled fsspec-2025.12.0━━━━━━━━━━━━━━━  3/15 [pyarrow]
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15/15 [peft]2m14/1

Load the dataset from hugging face https://huggingface.co/datasets/169Pi/indian_law

In [5]:
from datasets import load_dataset
from datasets import Dataset

In [6]:
dataset = load_dataset("169Pi/indian_law", split = "train")

In [7]:
dataset[0]

{'prompt': 'Explain the legal provisions relating to biodiversity conservation under the Biological Diversity Act, 2002. How does the National Biodiversity Authority (NBA) regulate access to biological resources in India?',
 'complex_cot': "What are the legislations impacting biodiversity and the environment in India in the last five years? How does informed consent relate to biosecurity in environmental governance?\n\nTo cover these areas, I need to use appropriate academic sources, peer-reviewed articles, and legal documents. Let me think about how each of these areas is connected. First, the Biological Diversity Act, 2002 - I remember that it was a significant piece of legislation in India to address biodiversity conservation. It likely includes provisions on protected areas, conservation measures, and penalties for illegal activities related to biodiversity.\n\nThen, the role of the National Biodiversity Authority (NBA) - I recall that the NBA is a regulatory body established under

In [9]:
for i in range(3):
    data = dataset[i]
    prompt = data["prompt"]
    response = data["response"]

    print(f"\n--- Sample {i+1} ---")
    print("Prompt:", prompt)
    print("Response:", response)



--- Sample 1 ---
Prompt: Explain the legal provisions relating to biodiversity conservation under the Biological Diversity Act, 2002. How does the National Biodiversity Authority (NBA) regulate access to biological resources in India?
Response: **Understanding Biodiversity Conservation in India: An Overview**

India has established a robust legal framework to protect its biodiversity, recognizing the importance of biological diversity to ecological health and economic sustainability. The Biological Diversity Act, 2002, is a cornerstone legislation that addresses biodiversity conservation through provisions related to protected areas, conservation measures, and penalizing illegal activities. This act also established the National Biodiversity Authority (NBA) as a regulatory body to oversee biodiversity-related matters.

**Role of the National Biodiversity Authority (NBA)**

The NBA, established under the Biological Diversity Act, plays a crucial role in regulating access to biological 

The dataset is in the format

> {prompt : " ",
complex_cot : " ",
response : " "}

We need to re-format the data in


> {
  "instruction": " ",
  "input": " ",
  "output": " "
} style





In [10]:
# Re-formatting the dataset in required format

def to_instruction_format(example):
    return {
        "instruction": (
            "Explain the following from a Company Secretary compliance "
            "perspective under Indian law"
        ),
        "input": example["prompt"],
        "output": example["response"]
    }



In [11]:
cs_dataset = dataset.map(to_instruction_format)

In [12]:
cs_dataset

Dataset({
    features: ['prompt', 'complex_cot', 'response', 'instruction', 'input', 'output'],
    num_rows: 47789
})

In [13]:
for i in range(3):
    data = cs_dataset[i]
    instruction = data["instruction"]
    input = data["input"]
    output = data["output"]

    print(f"\n--- Sample {i+1} ---")
    print("Instruction:", instruction)
    print("Input:", input)
    print("Output:", output)


--- Sample 1 ---
Instruction: Explain the following from a Company Secretary compliance perspective under Indian law
Input: Explain the legal provisions relating to biodiversity conservation under the Biological Diversity Act, 2002. How does the National Biodiversity Authority (NBA) regulate access to biological resources in India?
Output: **Understanding Biodiversity Conservation in India: An Overview**

India has established a robust legal framework to protect its biodiversity, recognizing the importance of biological diversity to ecological health and economic sustainability. The Biological Diversity Act, 2002, is a cornerstone legislation that addresses biodiversity conservation through provisions related to protected areas, conservation measures, and penalizing illegal activities. This act also established the National Biodiversity Authority (NBA) as a regulatory body to oversee biodiversity-related matters.

**Role of the National Biodiversity Authority (NBA)**

The NBA, establi

In [10]:
# Trimming the answers below 2000 chars to save from OOM errors
MAX_LEN = 2000

def trim_output(example):
    example["output"] = example["output"][:MAX_LEN]
    return example


In [11]:
cs_dataset = cs_dataset.map(trim_output)

Map:   0%|          | 0/47789 [00:00<?, ? examples/s]

In [12]:
cs_dataset

Dataset({
    features: ['prompt', 'complex_cot', 'response', 'instruction', 'input', 'output'],
    num_rows: 47789
})

Removing unnecessary columns

In [13]:
columns_remove = ["prompt", "complex_cot", "response"]

cs_dataset = cs_dataset.remove_columns(columns_remove)

In [14]:
cs_dataset[0]

{'instruction': 'Explain the following from a Company Secretary compliance perspective under Indian law',
 'input': 'Explain the legal provisions relating to biodiversity conservation under the Biological Diversity Act, 2002. How does the National Biodiversity Authority (NBA) regulate access to biological resources in India?',
 'output': "**Understanding Biodiversity Conservation in India: An Overview**\n\nIndia has established a robust legal framework to protect its biodiversity, recognizing the importance of biological diversity to ecological health and economic sustainability. The Biological Diversity Act, 2002, is a cornerstone legislation that addresses biodiversity conservation through provisions related to protected areas, conservation measures, and penalizing illegal activities. This act also established the National Biodiversity Authority (NBA) as a regulatory body to oversee biodiversity-related matters.\n\n**Role of the National Biodiversity Authority (NBA)**\n\nThe NBA, est

Defining a Tokenizer

In [14]:
from transformers import AutoTokenizer

In [15]:
model_name = "mistralai/Mistral-7B-v0.1"

In [16]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

Formatting the prompt

In [17]:
def format_prompt(example):
    return (
        f"Instruction:\n{example['instruction']}\n\n"
        f"Input:\n{example['input']}\n\n"
        f"Response:\n{example['output']}"
    )

Mapping the tokenizer

In [19]:
# def map_dataset(example) :
#   text = format_prompt(example)
#   return tokenizer(text,
#                    truncation = True,
#                    max_length = 1024,
#                    return_overflowing_tokens = True)

Applying the tokenizer

In [20]:
# tokenized_cs_dataset = cs_dataset.map(
#     map_dataset,
#     remove_columns = list(cs_dataset.columns)
# )

In [21]:
cs_dataset

Dataset({
    features: ['instruction', 'input', 'output'],
    num_rows: 47789
})

Define DataCollatorForLanguageModelling

In [18]:
from transformers import DataCollatorForLanguageModeling

In [23]:
# data_collator = DataCollatroForLanguageModelling.from_pretrained(
#     tokenizer = tokenizer,
#     mlm = False
# )

Define BitsAndBytesConfig

In [24]:
!pip uninstall -y bitsandbytes
!pip install -U bitsandbytes==0.43.3
!pip install -U transformers accelerate peft trl

Found existing installation: bitsandbytes 0.49.0
Uninstalling bitsandbytes-0.49.0:
  Successfully uninstalled bitsandbytes-0.49.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 MB 224.7 MB/s  0:00:000:00:0100:01


In [19]:
import torch
from transformers import BitsAndBytesConfig

In [20]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit = True,
    bnb_4bit_quant_type = "nf4",
    bnb_4bit_compute_dtype = torch.float16,
    bnb_4bit_use_double_quant = True
)

Apply BitsAndBytesConfig

In [27]:
!pip install -U bitsandbytes

  Using cached bitsandbytes-0.49.0-py3-none-manylinux_2_24_x86_64.whl.metadata (10 kB)
Using cached bitsandbytes-0.49.0-py3-none-manylinux_2_24_x86_64.whl (59.1 MB)
  Attempting uninstall: bitsandbytes
    Found existing installation: bitsandbytes 0.43.3
    Uninstalling bitsandbytes-0.43.3:
      Successfully uninstalled bitsandbytes-0.43.3


In [21]:
from transformers import AutoModelForCausalLM

In [27]:
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config = bnb_config,
    device_map = "auto",
    trust_remote_code = True
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Prepare model for K bit training

In [54]:
from peft import prepare_model_for_kbit_training

In [55]:
model = prepare_model_for_kbit_training(model)

In [56]:
for name, module in model.named_modules() :
  if "proj" in name.lower() :
    print(name)

base_model.model.base_model.model.base_model.model.model.layers.0.self_attn.q_proj
base_model.model.base_model.model.base_model.model.model.layers.0.self_attn.q_proj.base_layer
base_model.model.base_model.model.base_model.model.model.layers.0.self_attn.q_proj.lora_dropout
base_model.model.base_model.model.base_model.model.model.layers.0.self_attn.q_proj.lora_dropout.default
base_model.model.base_model.model.base_model.model.model.layers.0.self_attn.q_proj.lora_A
base_model.model.base_model.model.base_model.model.model.layers.0.self_attn.q_proj.lora_A.default
base_model.model.base_model.model.base_model.model.model.layers.0.self_attn.q_proj.lora_B
base_model.model.base_model.model.base_model.model.model.layers.0.self_attn.q_proj.lora_B.default
base_model.model.base_model.model.base_model.model.model.layers.0.self_attn.q_proj.lora_embedding_A
base_model.model.base_model.model.base_model.model.model.layers.0.self_attn.q_proj.lora_embedding_B
base_model.model.base_model.model.base_model.mo

Define LoraConfig

In [43]:
from peft import LoraConfig

In [44]:
lora_config = LoraConfig(
    r = 16,
    lora_alpha = 32,
    # target_modules = ["self_attn.q_proj", "self_attn.k_proj", "self_attn.v_proj", "self_attn.o_proj"],
    target_modules = "all-linear",
    lora_dropout = 0.1,
    bias = "none",
    task_type = "CAUSAL_LM"
)

Apply LoraConfig

In [49]:
print(type(model))

<class 'peft.peft_model.PeftModelForCausalLM'>


In [50]:
from peft import get_peft_model

In [51]:
model = get_peft_model(model, lora_config)

In [52]:
model.print_trainable_parameters()

trainable params: 42,520,576 || all params: 7,284,252,672 || trainable%: 0.5837


Total Trainable Parameters = 0.28875637 % and LoRA adapters are successfully attached

In [53]:
(20971520/7262703616)*100

0.2887563792882719

Training Config for model training

In [40]:
from transformers import TrainingArguments

In [41]:
# training_args = TrainingArguments(
#     output_dir = "./cs_lora_output_model",

#     # Training Hyperparameter
#     num_train_epochs = 5,
#     per_device_train_batch_size = 1,
#     per_device_eval_batch_size = 1,
#     gradient_accumulation_steps = 4,

#     # Optimization
#     learning_rate = 2e-4,
#     lr_scheduler_type = "cosine",
#     warmup_ratio = 0.05,
#     weight_decay = 0.01,

#     # Model Optimization
#     gradient_checkpointing = False,
#     optim = "paged_adamw_8bit",
#     max_grad_norm = 0.3,

#     # Logging
#     logging_dir = "./logs",
#     logging_steps = 100,

#     # Evaluation
#     eval_strategy = "steps",
#     eval_steps = 100,
#     save_strategy = "steps",
#     save_steps = 100,
#     save_total_limit = 2,

#     # Performance
#     fp16 = True,
#     report_to = "none"
# )

Applying training arguments to SFT trainer

In [42]:
from trl import SFTTrainer
from trl import SFTConfig

In [43]:
def build_text(example):
    return {
        "text": (
            f"Instruction:\n{example['instruction']}\n\n"
            f"Input:\n{example['input']}\n\n"
            f"Response:\n{example['output']}"
        )
    }

cs_dataset = cs_dataset.map(build_text)

Map:   0%|          | 0/47789 [00:00<?, ? examples/s]

In [44]:
cs_dataset

Dataset({
    features: ['instruction', 'input', 'output', 'text'],
    num_rows: 47789
})

In [45]:
sft_config = SFTConfig(
    # Output
    output_dir = "./cs_lora_output_model",

    dataset_text_field = "text",

    # Training Hyperparameters
    num_train_epochs = 2,
    per_device_train_batch_size = 1,
    per_device_eval_batch_size = 1,
    gradient_accumulation_steps = 8,

    # Optimization
    learning_rate = 5e-5,
    lr_scheduler_type = "cosine",
    warmup_ratio = 0.1,
    weight_decay = 0.01,
    optim = "paged_adamw_8bit",

    # Memory Optimization
    gradient_checkpointing = True,
    gradient_checkpointing_kwargs = {"use_reentrant": False},
    max_grad_norm = 1.0,

    # Logging
    logging_dir = "./logs",
    logging_steps = 100,

    # Evaluation
    eval_strategy = "no",
    # eval_strategy = "steps",
    # eval_steps = 100,
    save_strategy = "steps",
    save_steps = 500,
    save_total_limit = 50,

    # Performance
    bf16 = True,
    report_to = "none",

    # SFT specific settings
    max_length = 1024,
    packing = False,
)

In [46]:
trainer = SFTTrainer(
    model = model,
    args = sft_config,
    train_dataset = cs_dataset,
    processing_class = tokenizer
    # formatting_func = format_prompt,
)

Adding EOS to train dataset:   0%|          | 0/47789 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/47789 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/47789 [00:00<?, ? examples/s]

Training the model

In [47]:
print("Starting Training...")
trainer.train()

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'pad_token_id': 2}.


Starting Training...


Step,Training Loss
100,1.463400
200,1.181200
300,1.128400
400,1.080300
500,1.068400
600,1.054500
700,1.034400
800,1.028600
900,1.032200
1000,1.018500


KeyboardInterrupt: 

In [37]:
from peft import PeftModel

In [38]:
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto"
)

model = PeftModel.from_pretrained(
    base_model,
    "./cs_lora_output_model/checkpoint-1500"
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [39]:
from transformers import pipeline

model_name = "mistralai/Mistral-7B-v0.1"   # same base model you trained on
checkpoint_path = "./cs_lora_output_model/checkpoint-1500"

# Tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token

# Base model (quantized)
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto"
)

# Load LoRA checkpoint
model = PeftModel.from_pretrained(
    base_model,
    checkpoint_path
)

model.eval()


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

PeftModelForCausalLM(
  (base_model): LoraModel(
    (model): MistralForCausalLM(
      (model): MistralModel(
        (embed_tokens): Embedding(32000, 4096)
        (layers): ModuleList(
          (0-31): 32 x MistralDecoderLayer(
            (self_attn): MistralAttention(
              (q_proj): lora.Linear4bit(
                (base_layer): Linear4bit(in_features=4096, out_features=4096, bias=False)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=4096, out_features=16, bias=False)
                )
                (lora_B): ModuleDict(
                  (default): Linear(in_features=16, out_features=4096, bias=False)
                )
                (lora_embedding_A): ParameterDict()
                (lora_embedding_B): ParameterDict()
                (lora_magnitude_vector): ModuleDict()
              )
              (k_proj

In [40]:
# same output formatter as during training

def build_prompt(instruction, input_text=""):
    if input_text.strip():
        return (
            f"Instruction:\n{instruction}\n\n"
            f"Input:\n{input_text}\n\n"
            f"Response:\n"
        )
    else:
        return (
            f"Instruction:\n{instruction}\n\n"
            f"Response:\n"
        )


In [41]:
text_generator = pipeline(
    task="text-generation",
    model=model,
    tokenizer=tokenizer,
    device_map="auto"
)

Device set to use cuda:0


#### Test the output

In [70]:
prompt = build_prompt(
    # "Explain the role and responsibilities of a Company Secretary under the Companies Act, 2013."
    # "Explain the concept of consideration in contract law with an example."
    # "Can a contract be valid without consideration? Explain with case reasoning."
    "A buyer signs a contract under coercion. Step by step, explain whether the contract is enforceable."
)

outputs = text_generator(
    prompt,
    max_new_tokens=512,
    temperature=0.2,
    top_p=0.9,
    do_sample=True,
    repetition_penalty=1.1,
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.eos_token_id
)

print(outputs[0]["generated_text"])


Instruction:
A buyer signs a contract under coercion. Step by step, explain whether the contract is enforceable.

Response:
The contract is not enforceable. The contract was signed under duress. Duress is defined as “the use of force or threats to compel another person to act against his will.” Restatement (Second) of Contracts § 176(2). In this case, the seller threatened to kill the buyer if he did not sign the contract. This threat constitutes duress because it is an illegal threat that would cause a reasonable person to act against his will. See id. at § 176 cmt. b. Because the buyer acted under duress, the contract is unenforceable. Id. at § 178.


In [71]:
def extract_response(generated_text):
    return generated_text.split("Response:")[-1].strip()

final_answer = extract_response(outputs[0]["generated_text"])
print(final_answer)


The contract is not enforceable. The contract was signed under duress. Duress is defined as “the use of force or threats to compel another person to act against his will.” Restatement (Second) of Contracts § 176(2). In this case, the seller threatened to kill the buyer if he did not sign the contract. This threat constitutes duress because it is an illegal threat that would cause a reasonable person to act against his will. See id. at § 176 cmt. b. Because the buyer acted under duress, the contract is unenforceable. Id. at § 178.


In [72]:
!pip install streamlit

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 72.6 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 795.4/795.4 kB 11.9 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 216.4 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11/11 [streamlit]11 [streamlit]


In [ ]:
!streamlit run app.py

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)





  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://10.96.0.107:8501
  External URL: http://204.12.163.123:8501

────────────────────────── Traceback (most recent call last) ───────────────────────────
  /home/zeus/miniconda3/envs/cloudspace/lib/python3.12/site-packages/huggingface_hub/u  
  tils/_http.py:402 in hf_raise_for_status                                              
                                                                                        
    399 │   >         If request failed for a reason not listed above.                  
    400 │   """                                                                         
    401 │   try:                                                                        
  ❱ 402 │   │   response.raise_for_status()                                             
    403 │   except HTTPError as e:                                                      
    404 │   │   error_

In [ ]:
from transformers import AutoModelForCausalLM
from peft import PeftModel
import torch

BASE_MODEL = "mistralai/Mistral-7B-v0.1"
CHECKPOINT = "./cs_lora_output_model/checkpoint-1500"
EXPORT_DIR = "./lora_adapters_final"

# Load base model
base_model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    device_map="auto",
    torch_dtype=torch.float16
)

# Load LoRA from checkpoint
model = PeftModel.from_pretrained(base_model, CHECKPOINT)

# Save adapters in inference-ready format
model.save_pretrained(EXPORT_DIR)

print("✅ LoRA adapters exported to:", EXPORT_DIR)


In [ ]:
! ls ./lora_adapters_final